In [25]:
# Imports for scraping
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import datetime

# Imports for data formatting
import uuid
import hashlib
import datetime
import json

In [30]:
TODAY = datetime.date.today().strftime("%Y-%m-%d")

def generate_json(author, text, date_created):
    output = {}
    # get doc id
    m = hashlib.md5()
    m.update(text.encode('utf-8'))
    output['documentID'] = str(uuid.UUID(m.hexdigest())) 

    # get authorID
    m = hashlib.md5()
    m.update(author.encode('utf-8'))
    output['authorIDs'] = [str(uuid.UUID(m.hexdigest()))]

    output['fullText'] = text
    output["spanAttribution"] = [{"authorID":output['authorIDs'][0],
                                    "start":0,
                                    "end":len(text)}]
    output["isNeedle"] = False
    output["collectionNum"] = "HRS 1"
    output["source"] = 'https://www.rottentomatoes.com/'
    output["dateCollected"] = TODAY
    output["dateCreated"] = datetime.datetime.strptime(date_created, "%b %d, %Y").strftime("%Y-%m-%d")
    output["publiclyAvailable"] = True
    output["deidentified"] = False
    output["languages"] = ["en"]
    output["lengthWords"] = len(text.split(' '))
    output["sourceSpecific"] = {"authorName": author}
    print(output)
    return output

In [31]:
critic = 'scott-tobias'
driver = webdriver.Chrome()
page_url = f'https://www.rottentomatoes.com/critics/{critic}/movies/'
driver.get(page_url)
reviews = []
# amount of time to wait for 
next_wait = WebDriverWait(driver, 4)
table_wait = WebDriverWait(driver, 10)
for i in range(1):
    # extract reviews section from each page
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'html.parser')
    table = soup.find('table', {'data-qa': 'critic-reviews-table'})
    if table:
        # Find all rows in the table's tbody
        rows = table.find('tbody').find_all('tr', {'data-qa': 'row'})

        # Iterate through the rows to extract the review text
        for row in rows:
            # Find the review's td element
            review_td = row.find('td', {'data-qa': 'critic-review'})
            
            # Check if the review_td is found
            if review_td:
                # Extract the review text
                review_text = review_td.find('span').text.strip()
                date_created = review_td.find('div').find('span').text
                generate_json(critic, review_text, date_created)
                break
                # if date_span:
                #     review_date = date_span.find_next('span').text.strip()
                # else:
                #     review_date = None
                # print(review_date)
    try:
        next_button = next_wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'rt-button.next'))
        )
        # Click the "Next" button
        next_button.click()
        # Wait for the table's data to change
        table_wait.until(EC.staleness_of(driver.find_element(By.CSS_SELECTOR, 'table[data-qa="critic-reviews-table"]')))
        time.sleep(0.2)
                    
    except Exception as e:
        break
        
    # Close the WebDriver when done
    driver.quit()
    print("reviews found for critic " + critic + ": " + str(len(reviews)))


{'documentID': 'aa271a03-92d7-1136-180f-091c4a7ee860', 'authorIDs': ['e8b741bd-3d2b-b070-ff82-f70563fa36e3'], 'fullText': "There are plenty of boys-to-men films about the raunchy adventures of pleasure-seeking teenagers, but Alfonso Cuarón's inspired and exhilarating road movie... taps so directly into their teeming energy that it all but levitates off the screen.", 'spanAttribution': [{'authorID': 'e8b741bd-3d2b-b070-ff82-f70563fa36e3', 'start': 0, 'end': 243}], 'isNeedle': False, 'collectionNum': '', 'source': 'https://www.rottentomatoes.com/', 'dateCollected': '2023-09-21', 'dateCreated': '2023-09-09', 'publiclyAvailable': True, 'deidentified': False, 'languages': ['en'], 'lengthWords': 36, 'sourceSpecific': {'authorName': 'scott-tobias'}}
reviews found for critic scott-tobias: 0
